In [1]:
# Import torch & Check CUDA availability
import torch

print(torch.cuda.is_available())
print(torch.cuda.device_count())
print(torch.cuda.current_device())

True
1
0


In [2]:
# Get CUDA device name
print(torch.cuda.device(0))
print(torch.cuda.get_device_name(0))

NVIDIA A30


In [3]:
from torch_geometric.datasets import AmazonProducts

# Import dataset from PyTorch Geometric
dataset = AmazonProducts(root="/dfs6/pub/seminl1/AmazonProducts")
data = dataset[0]

# Store the dataset to GPU
#data = data.pin_memory()
#data = data.to('cuda:0', non_blocking=True)

# Print information about the dataset
print(f'Dataset: {dataset}')
print('-------------------')
print(f'Number of graphs: {len(dataset)}')
print(f'Number of nodes: {data.x.shape[0]}')
print(f'Number of features: {dataset.num_features}')
print(f'Number of classes: {dataset.num_classes}')

# Print information about the graph
#print(f'\nGraph:')
#print('------')
#print(f'Edges are directed: {data.is_directed()}')
#print(f'Graph has isolated nodes: {data.has_isolated_nodes()}')
#print(f'Graph has loops: {data.has_self_loops()}')

Dataset: AmazonProducts()
-------------------
Number of graphs: 1
Number of nodes: 1569960
Number of features: 200
Number of classes: 107


In [4]:
# Check whether the dataset is stored on the GPU or not
print(f'Graph is stored on the GPU: {data.is_cuda}')

Graph is stored on the GPU: False


In [5]:
# Print first element
print(f'Graph: {data}')

Graph: Data(x=[1569960, 200], edge_index=[2, 264339468], y=[1569960, 107], train_mask=[1569960], val_mask=[1569960], test_mask=[1569960])


In [6]:
# Node feature matrix information
print(f'x = {data.x.shape}')
print(data.x)

x = torch.Size([1569960, 200])
tensor([[-0.1466,  0.2226, -0.3597,  ...,  0.1699,  0.8974,  1.6527],
        [-0.2805,  0.0190,  0.4301,  ..., -1.1758, -1.8365, -1.1693],
        [ 0.2554,  0.2519, -0.0291,  ...,  1.3751, -0.0735,  0.6262],
        ...,
        [-0.8121,  0.3626, -0.7781,  ...,  0.0639,  0.8645,  0.0389],
        [ 1.5977, -2.3989, -0.0569,  ..., -1.4413,  0.2966,  0.0985],
        [-0.1663,  0.0629, -0.0474,  ...,  0.1853, -0.1216, -0.9181]])


In [7]:
# Edge index information
print(f'edge_index = {data.edge_index.shape}')
print(data.edge_index)

edge_index = torch.Size([2, 264339468])
tensor([[      0,       0,       0,  ..., 1569958, 1569959, 1569959],
        [      0,   83053,  210735,  ..., 1569958, 1178338, 1569959]])


In [8]:
# Try to get the number of addition operation
start_point = 0
numAddition = 0
numEdges = 264339468
for i in range(numEdges-1):
    if data.edge_index[0][i+1] == start_point:
        numAddition += 1
    else:
        start_point = data.edge_index[0][i+1]
        if data.edge_index[0][i] % 10000 == 0:
            print(data.edge_index[0][i])
print(numAddition)

tensor(0)
tensor(10000)
tensor(20000)
tensor(30000)
tensor(40000)
tensor(50000)
tensor(60000)
tensor(70000)
tensor(80000)
tensor(90000)
tensor(100000)
tensor(110000)
tensor(120000)
tensor(130000)
tensor(140000)
tensor(150000)
tensor(160000)
tensor(170000)
tensor(180000)
tensor(190000)
tensor(200000)
tensor(210000)
tensor(220000)
tensor(230000)
tensor(240000)
tensor(250000)
tensor(260000)
tensor(270000)
tensor(280000)
tensor(290000)
tensor(300000)
tensor(310000)
tensor(320000)
tensor(330000)
tensor(340000)
tensor(350000)
tensor(360000)
tensor(370000)
tensor(380000)
tensor(390000)
tensor(400000)
tensor(410000)
tensor(420000)
tensor(430000)
tensor(440000)
tensor(450000)
tensor(460000)
tensor(470000)
tensor(480000)
tensor(490000)
tensor(500000)
tensor(510000)
tensor(520000)
tensor(530000)
tensor(540000)
tensor(550000)
tensor(560000)
tensor(570000)
tensor(580000)
tensor(590000)
tensor(600000)
tensor(610000)
tensor(620000)
tensor(630000)
tensor(640000)
tensor(650000)
tensor(660000)
tensor(67

In [9]:
# Ground-truth labels
print(f'y(one-hot) = {data.y.shape}')
print(data.y)
data.y = torch.argmax(data.y, dim=1)
print(f'y(label) = {data.y.shape}')
print(data.y)

y(one-hot) = torch.Size([1569960, 107])
tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]])
y(label) = torch.Size([1569960])
tensor([43, 37, 76,  ..., 99, 14, 80])


In [10]:
# Train mask
print(f'train_mask = {data.train_mask.shape}')
print(data.train_mask)

train_mask = torch.Size([1569960])
tensor([ True,  True, False,  ..., False,  True,  True])


In [11]:
from torch_geometric.loader import NeighborLoader

# Create batches with neighbor sampling
train_loader = NeighborLoader(
    data,
    num_neighbors=[-1],
    batch_size=1024,
    input_nodes=data.train_mask,
)

In [12]:
# Create a simple GCN with only one GCN layer
import torch.nn.functional as F

from torch.nn import Linear
from torch_geometric.nn import GCNConv

class GCN(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.gcn1 = GCNConv(dataset.num_features, dataset.num_classes)
        self.optimizer = torch.optim.Adam(self.parameters(),
                                          lr=0.01,
                                          weight_decay=5e-4)

    def forward(self, x, edge_index):
        x = self.gcn1(x, edge_index)
        z = F.log_softmax(x, dim=1)
        return x, z

In [13]:
def accuracy(pred_y, y):
    """Calculate accuracy."""
    return ((pred_y == y).sum() / len(y)).item()

def train(model, data, train_loader, device):
    """Train a GNN model and return the trained model."""
    criterion = torch.nn.CrossEntropyLoss()
    optimizer = model.optimizer
    epochs = 200

    model.train()
    for epoch in range(epochs+1):
        # Training
        total_loss = 0
        acc = 0
        val_loss = 0
        val_acc = 0
        
        # Train on batches
        for batch in train_loader:
            batch = batch.to(device)
            optimizer.zero_grad()
            h, out = model(batch.x, batch.edge_index)
            loss = criterion(out[batch.train_mask], batch.y[batch.train_mask])
            total_loss += loss
            acc += accuracy(out[batch.train_mask].argmax(dim=1), batch.y[batch.train_mask])
            loss.backward()
            optimizer.step()

        # Print metrics every 10 epochs
        if(epoch % 10 == 0):
            print(f'Epoch {epoch:>3} | Train Loss: {total_loss/len(train_loader):.3f} | Train Acc: '
                  f'{acc/len(train_loader)*100:>6.2f}%')
          
    return model, h, out

In [14]:
import time

# Create GCN model
gcn = GCN()
print(gcn)
print()

# Train and test
# Train
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

start_time = time.process_time()
gcn_model, gcn_output, final_output = train(gcn.to(device), data.to(device), train_loader, device)
end_time = time.process_time()
elapsed_time = end_time - start_time
print()
print('Elapsed Time(CPU): ', elapsed_time, 'seconds')

GCN(
  (gcn1): GCNConv(200, 107)
)

Epoch   0 | Train Loss: 2.052 | Train Acc:  53.38%
Epoch  10 | Train Loss: 1.876 | Train Acc:  54.70%
Epoch  20 | Train Loss: 1.876 | Train Acc:  54.70%
Epoch  30 | Train Loss: 1.876 | Train Acc:  54.70%
Epoch  40 | Train Loss: 1.876 | Train Acc:  54.70%
Epoch  50 | Train Loss: 1.876 | Train Acc:  54.70%
Epoch  60 | Train Loss: 1.876 | Train Acc:  54.70%
Epoch  70 | Train Loss: 1.876 | Train Acc:  54.70%
Epoch  80 | Train Loss: 1.876 | Train Acc:  54.70%
Epoch  90 | Train Loss: 1.876 | Train Acc:  54.70%
Epoch 100 | Train Loss: 1.876 | Train Acc:  54.70%
Epoch 110 | Train Loss: 1.876 | Train Acc:  54.70%
Epoch 120 | Train Loss: 1.876 | Train Acc:  54.70%
Epoch 130 | Train Loss: 1.876 | Train Acc:  54.70%
Epoch 140 | Train Loss: 1.876 | Train Acc:  54.70%
Epoch 150 | Train Loss: 1.876 | Train Acc:  54.70%
Epoch 160 | Train Loss: 1.876 | Train Acc:  54.70%
Epoch 170 | Train Loss: 1.876 | Train Acc:  54.70%
Epoch 180 | Train Loss: 1.876 | Train Acc:  54

In [15]:
print(device)

cuda
